In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import re
from pathlib import Path
from typing import Union, Sequence, Optional
import random

## Run Gromacs Simulation

In [ ]:
!printf "6\n2\n" | gmx pdb2gmx -f 1AKI.pdb -o lysozyme_processed.gro
!gmx editconf -f lysozyme_processed.gro -o boxed.gro -c -d 1.5 -bt cubic
!gmx solvate -cp boxed.gro -cs tip4p.gro -o solvated.gro -p topol.top
!gmx grompp -f ions.mdp -c solvated.gro -p topol.top -o ions.tpr -maxwarn 1
!printf "13\n0\n" |gmx genion -s ions.tpr -o solv_ions.gro -p topol.top -neutral -conc 0.15
!gmx grompp -f minim.mdp -c solv_ions.gro -p topol.top -o em.tpr
!gmx mdrun -deffnm em
!gmx grompp -f nvt.mdp -c em.gro -p topol.top -o nvt.tpr
!gmx mdrun -deffnm nvt
!gmx grompp -f npt.mdp -c nvt.gro -p topol.top -o npt.tpr
!gmx mdrun -deffnm npt
!gmx grompp -f md.mdp -c npt.gro -p topol.top -o md.tpr
!gmx mdrun -deffnm md

                     :-) GROMACS - gmx pdb2gmx, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/htchi/gromacs-2023.2/Simulations/Proteins/Lysozyme
Command line:
  gmx pdb2gmx -f 1AKI.pdb -o lysozyme_processed.gro

Select the Force Field:

From '/usr/local/gromacs/share/gromacs/top':

 1: AMBER03 protein, nucleic AMBER94 (Duan et al., J. Comp. Chem. 24, 1999-2012, 2003)

 2: AMBER94 force field (Cornell et al., JACS 117, 5179-5197, 1995)

 3: AMBER96 protein, nucleic AMBER94 (Kollman et al., Acc. Chem. Res. 29, 461-469, 1996)

 4: AMBER99 protein, nucleic AMBER94 (Wang et al., J. Comp. Chem. 21, 1049-1074, 2000)

 5: AMBER99SB protein, nucleic AMBER94 (Hornak et al., Proteins 65, 712-725, 2006)

 6: AMBER99SB-ILDN protein, nucleic AMBER94 (Lindorff-Larsen et al., Proteins 78, 1950-58, 2010)

 7: AMBERGS force field (Garcia & Sanbonmatsu, PNAS 99, 2782-2787, 2002)

 8: CHARMM27 all-atom force field (CHARM22 plus CMAP for proteins)

In [27]:
! printf "2\n0\n" | gmx trjconv -s md.tpr -f md.xtc -o md_whole.xtc -pbc mol -ur compact -center 
! printf "4\n0\n" | gmx energy -f md.edr -o potential_energy.xvg
! printf "8\n0\n" | gmx energy -f md.edr -o temperature.xvg
! printf "5\n0\n" | gmx energy -f md.edr -o kinetic_energy.xvg
! printf "7\n0\n" | gmx energy -f md.edr -o conserved_energy.xvg
! printf "6\n0\n" | gmx energy -f md.edr -o total_energy.xvg
! printf "9\n0\n" | gmx energy -f md.edr -o pressure.xvg

                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/htchi/gromacs-2023.2/Simulations/Proteins/Lysozyme
Command line:
  gmx trjconv -s md.tpr -f md.xtc -o md_whole.xtc -pbc mol -ur compact -center

Note that major changes are planned in future for trjconv, to improve usability and utility.
Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file md.tpr, VERSION 2023.2 (single precision)
Reading file md.tpr, VERSION 2023.2 (single precision)
Select group for centering
Group     0 (         System) has 67284 elements
Group     1 (        Protein) has  1960 elements
Group     2 (      Protein-H) has  1001 elements
Group     3 (        C-alpha) has   129 elements
Group     4 (       Backbone) has   387 elements
Group     5 (      MainChain) has   517 elements
Group     6 (   MainChain+Cb) has   634 elements
Group     7 (    MainChain+H) has   646 elements
Group     8 ( 